In [1]:
import numpy as np
import pandas as pd
import csv
import cv2
import matplotlib.pylab as plt
import sys
import math
import os
from PIL import Image , ImageDraw

In [75]:
#- hair and small black marks removal
images = []
with open('images-diseases.csv' , encoding='utf-8') as data:
 reader = csv.DictReader(data)
 for row in reader:
  images.append(row['image'])
    
SIZE=256 #for less resources and best for the CNN model
for imag in images:
  image = 'Dataset/' + imag + '.jpg'
  img = cv2.imread(image) 
  resized_imgs = cv2.resize(img,(SIZE,SIZE))
  N_images = resized_imgs
  
  gray = []
  grayScale = cv2.cvtColor(resized_imgs, cv2.COLOR_BGR2GRAY)
  kernel = cv2.getStructuringElement(1,(17,17))
  blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
  ret,thresh2 = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
	
  dst = cv2.inpaint(resized_imgs, thresh2, 1 ,cv2.INPAINT_TELEA)
  cv2.imwrite('result/'+imag+'.jpg', dst, [int(cv2.IMWRITE_JPEG_QUALITY), 90]) 

In [ ]:
#- contrast enhancement
images = []
with open('images-diseases.csv' , encoding='utf-8') as data:
  reader = csv.DictReader(data)
  for row in reader:
    images.append(row['image'])
    
N_images = ["ISIC_0000014"]
for imag in images:
  image = 'result/' + imag + '.jpg'
  img = cv2.imread(image) 

  lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(lab)
  clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(5,5))
  cl = clahe.apply(l)
  limg = cv2.merge((cl,a,b))

  final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
  cv2.imwrite('contrast/'+ imag +'.jpg', final) 

In [ ]:
#- increase brightness
images = []
with open('images-diseases.csv' , encoding='utf-8') as data:
  reader = csv.DictReader(data)
  for row in reader:
    images.append(row['image'])

for imag in images:
  image = 'contrast/' + imag + '.jpg'
  img = cv2.imread(image) 
  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 

  h, s, v = cv2.split(hsv)
  v += 255 * 2
  final_hsv = cv2.merge((h, s, v))

  img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
  cv2.imwrite('brightness/'+ imag +'.jpg', img) 

In [8]:
#- remove vignette of all shapes under variable change
images = []
with open('images-diseases.csv' , encoding='utf-8') as data:
  reader = csv.DictReader(data)
  for row in reader:
    images.append(row['image'])

for imag in images:
  image = 'brightness/' + imag + '.jpg'
  img = cv2.imread(image)
  
  B_MIN = np.array([0, 0, 0], np.uint8)
  B_MAX = np.array([40, 40, 40], np.uint8)

  dst = cv2.inRange(img, B_MIN, B_MAX)
  no_b = cv2.countNonZero(dst)
  d = 5
  if no_b > d:    
   cv2.imwrite('vignette4/'+ imag +'.jpg', img) 

In [ ]:
#- test for annotation removal by color but it doesn't work
images = []
with open('images-diseases.csv' , encoding='utf-8') as data:
  reader = csv.DictReader(data)
  for row in reader:
    images.append(row['image'])

for imag in images:
  image = 'result/' + imag + '.jpg'
  img = cv2.imread(image)
  
  B_MIN = np.array([43,149,107], np.uint8)
  B_MAX = np.array([45,150,109], np.uint8)

  dst = cv2.inRange(img, B_MIN, B_MAX)
  no_b = cv2.countNonZero(dst)
  d = 50
  if no_b > d:    
   cv2.imwrite('annotation/'+ imag +'.jpg', img) 

In [23]:
#- for annotation ,rulers,marks removal and location crop
#- under varible change and test and visualization
names = []
names = os.listdir(('n8/'))

for imag in names:
  image = 'n8/' + imag
  img = cv2.imread(image)
  
  width, height = 206, 206
  x, y = 10, 40
  crop_img = img[y:y+height, x:x+width]
  crop_img = cv2.resize(crop_img,(256,256))
  
  cv2.imwrite('n8/'+ imag , crop_img, [int(cv2.IMWRITE_JPEG_QUALITY), 90]) 

In [ ]:
#- enhance contrast
  lab= cv2.cvtColor(crop_img, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(lab)
  clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(5,5))
  cl = clahe.apply(l)
  limg = cv2.merge((cl,a,b))
  final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
  
  #- brighten
  hsv = cv2.cvtColor(final, cv2.COLOR_BGR2HSV) #convert it to hsv

  h, s, v = cv2.split(hsv)
  v += 255 * 2
  final_hsv = cv2.merge((h, s, v))

  img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
  

In [77]:
#- tested for enhance contrast by equalization but didn't work

# i = cv2.imread('ISIC_0000000.jpg')
# gray = cv2.cvtColor(i,cv2.COLOR_RGB2GRAY)
# image_enhanced = cv2.equalizeHist(gray)
# image_enhanced = cv2.cvtColor(image_enhanced,cv2.COLOR_GRAY2RGB)
# cv2.imwrite("imag.jpg", image_enhanced)